# Module 0.1: 向量與矩陣基礎

這個 notebook 會帶你從零開始理解向量和矩陣，並實作基本運算。

## 學習目標

1. 理解向量的幾何意義
2. 內積 (dot product) 的計算和意義
3. 各種範數 (norm)
4. 矩陣乘法作為線性變換

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 12

---
## Part 1: 向量的直覺

### 什麼是向量？

向量可以用兩種方式理解：

1. **作為一個點的座標**：向量 `[3, 4]` 代表 2D 空間中的一個點
2. **作為方向和長度**：從原點出發，指向某個方向，有一定的長度

在機器學習中，我們常常把**資料表示成向量**：
- 一張 28×28 的灰階圖片 → 784 維向量
- 一個人的特徵（身高、體重、年齡）→ 3 維向量

In [ ]:
def plot_vector(v, origin=(0, 0), color='blue', label=None):
    """畫出從 origin 出發的向量"""
    plt.quiver(origin[0], origin[1], v[0], v[1], 
               angles='xy', scale_units='xy', scale=1, 
               color=color, label=label, width=0.02)

# 範例：畫幾個向量
v1 = np.array([3, 2])
v2 = np.array([1, 4])
v3 = v1 + v2  # 向量加法

plt.figure(figsize=(8, 8))
plot_vector(v1, color='blue', label=f'v1 = {v1}')
plot_vector(v2, color='red', label=f'v2 = {v2}')
plot_vector(v3, color='green', label=f'v1 + v2 = {v3}')
# 顯示向量加法的平行四邊形法則
plot_vector(v2, origin=v1, color='red', label=None)

plt.xlim(-1, 6)
plt.ylim(-1, 7)
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='k', linewidth=0.5)
plt.axvline(x=0, color='k', linewidth=0.5)
plt.legend()
plt.title('向量加法：平行四邊形法則')
plt.gca().set_aspect('equal')
plt.show()

---
## Part 2: 向量的長度 (Norm)

向量的「長度」有很多種定義方式：

### L2 Norm（歐幾里得距離）- 最常用！

$$||\mathbf{v}||_2 = \sqrt{v_1^2 + v_2^2 + \cdots + v_n^2} = \sqrt{\sum_{i=1}^n v_i^2}$$

這就是我們直覺上的「長度」，也是畢氏定理的推廣。

### L1 Norm（曼哈頓距離）

$$||\mathbf{v}||_1 = |v_1| + |v_2| + \cdots + |v_n| = \sum_{i=1}^n |v_i|$$

想像在格子狀的街道上走路，只能走直線（不能走對角線）。

### L∞ Norm（最大值範數）

$$||\mathbf{v}||_\infty = \max(|v_1|, |v_2|, \ldots, |v_n|)$$

### 為什麼要知道這些？

- **L2 正則化**（Ridge）使用 L2 norm
- **L1 正則化**（Lasso）使用 L1 norm，會產生稀疏解
- 不同的 norm 定義不同的「距離」概念

In [ ]:
# 視覺化不同 norm 的「單位圓」
# 單位圓 = 所有 norm = 1 的點的集合

theta = np.linspace(0, 2*np.pi, 1000)

# L2 單位圓（真正的圓）
x_l2 = np.cos(theta)
y_l2 = np.sin(theta)

# L1 單位圓（菱形）
# |x| + |y| = 1
x_l1 = np.concatenate([np.linspace(0, 1, 250), np.linspace(1, 0, 250),
                       np.linspace(0, -1, 250), np.linspace(-1, 0, 250)])
y_l1 = np.concatenate([1 - x_l1[:250], -1 + x_l1[250:500],
                       -1 - x_l1[500:750], 1 + x_l1[750:]])

# L∞ 單位圓（正方形）
# max(|x|, |y|) = 1
x_linf = np.array([1, 1, -1, -1, 1])
y_linf = np.array([1, -1, -1, 1, 1])

plt.figure(figsize=(8, 8))
plt.plot(x_l2, y_l2, 'b-', linewidth=2, label='L2 (圓)')
plt.plot(x_l1, y_l1, 'r-', linewidth=2, label='L1 (菱形)')
plt.plot(x_linf, y_linf, 'g-', linewidth=2, label='L∞ (正方形)')
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='k', linewidth=0.5)
plt.axvline(x=0, color='k', linewidth=0.5)
plt.legend(fontsize=12)
plt.title('不同 Norm 的「單位圓」')
plt.gca().set_aspect('equal')
plt.xlim(-1.5, 1.5)
plt.ylim(-1.5, 1.5)
plt.show()

### 練習 1: 實作各種 Norm

**提示**：
- L2: 先平方、再加總、最後開根號
- L1: 先取絕對值、再加總
- L∞: 取絕對值中的最大值

In [ ]:
def l2_norm(v):
    """
    計算向量的 L2 norm（歐幾里得長度）
    
    Parameters
    ----------
    v : np.ndarray
        輸入向量
    
    Returns
    -------
    float
        L2 norm
    
    Example
    -------
    >>> l2_norm(np.array([3, 4]))
    5.0
    """
    # 解答：
    return np.sqrt(np.sum(v ** 2))


def l1_norm(v):
    """
    計算向量的 L1 norm（曼哈頓距離）
    
    Example
    -------
    >>> l1_norm(np.array([3, -4]))
    7.0
    """
    # 解答：
    return np.sum(np.abs(v))


def linf_norm(v):
    """
    計算向量的 L∞ norm（最大絕對值）
    
    Example
    -------
    >>> linf_norm(np.array([3, -4, 2]))
    4.0
    """
    # 解答：
    return np.max(np.abs(v))

In [ ]:
# 測試
test_v = np.array([3, 4])

print(f"v = {test_v}")
print(f"L2 norm: {l2_norm(test_v)} (expected: 5.0)")
print(f"L1 norm: {l1_norm(test_v)} (expected: 7.0)")
print(f"L∞ norm: {linf_norm(test_v)} (expected: 4.0)")

# 和 numpy 比較驗證
print("\n✓ 驗證結果：")
assert np.isclose(l2_norm(test_v), np.linalg.norm(test_v, 2))
assert np.isclose(l1_norm(test_v), np.linalg.norm(test_v, 1))
assert np.isclose(linf_norm(test_v), np.linalg.norm(test_v, np.inf))
print("所有測試通過！")

---
## Part 3: 內積 (Dot Product)

兩個向量的內積定義為：

$$\mathbf{a} \cdot \mathbf{b} = a_1 b_1 + a_2 b_2 + \cdots + a_n b_n = \sum_{i=1}^n a_i b_i$$

### 內積的幾何意義（超重要！）

$$\mathbf{a} \cdot \mathbf{b} = ||\mathbf{a}|| \cdot ||\mathbf{b}|| \cdot \cos\theta$$

其中 θ 是兩向量的夾角。

這告訴我們：
- **內積 > 0**：兩向量夾角 < 90°（大致同方向）
- **內積 = 0**：兩向量**垂直**（正交）
- **內積 < 0**：兩向量夾角 > 90°（大致反方向）

### 內積作為「投影」

如果 b 是單位向量（||b|| = 1），則 a·b 就是 a 在 b 方向上的投影長度。

In [ ]:
# 視覺化內積與角度的關係
a = np.array([1, 0])  # 固定向量 a

angles = [0, 45, 90, 135, 180]
plt.figure(figsize=(12, 4))

for i, angle in enumerate(angles):
    plt.subplot(1, 5, i+1)
    
    # 創建角度為 angle 的向量 b
    theta = np.radians(angle)
    b = np.array([np.cos(theta), np.sin(theta)])
    
    # 計算內積
    dot = np.dot(a, b)
    
    plot_vector(a, color='blue', label='a')
    plot_vector(b, color='red', label='b')
    
    plt.xlim(-1.5, 1.5)
    plt.ylim(-0.5, 1.5)
    plt.grid(True, alpha=0.3)
    plt.gca().set_aspect('equal')
    plt.title(f'θ = {angle}°\na·b = {dot:.2f}')

plt.tight_layout()
plt.show()

### 練習 2: 實作內積和 Cosine Similarity

**Cosine Similarity** 是機器學習中非常常用的相似度度量：

$$\text{cosine\_similarity}(\mathbf{a}, \mathbf{b}) = \frac{\mathbf{a} \cdot \mathbf{b}}{||\mathbf{a}|| \cdot ||\mathbf{b}||}$$

這個值介於 -1 和 1 之間，只看方向，不看長度。

**提示**：
- 內積可以用 `np.sum(a * b)` 計算（元素相乘再加總）
- cosine similarity 就是內積除以兩個 L2 norm 的乘積

In [ ]:
def dot_product(a, b):
    """
    計算兩個向量的內積
    
    不要使用 np.dot 或 @
    """
    # 解答：
    return np.sum(a * b)


def cosine_similarity(a, b):
    """
    計算兩個向量的 cosine similarity
    
    cosine_similarity = (a · b) / (||a|| × ||b||)
    
    這個值介於 -1 和 1 之間：
    - 1: 完全同向
    - 0: 垂直
    - -1: 完全反向
    """
    # 解答：
    dot = dot_product(a, b)
    norm_a = l2_norm(a)
    norm_b = l2_norm(b)
    return dot / (norm_a * norm_b)

In [ ]:
# 測試
a = np.array([1, 0])
b = np.array([1, 1])
c = np.array([0, 1])
d = np.array([-1, 0])

print("測試內積：")
print(f"a·b = {dot_product(a, b)} (expected: 1)")
print(f"a·c = {dot_product(a, c)} (expected: 0)")
print(f"a·d = {dot_product(a, d)} (expected: -1)")

print("\n測試 cosine similarity：")
print(f"cosine(a, b) = {cosine_similarity(a, b):.4f} (expected: 0.7071 = cos(45°))")
print(f"cosine(a, c) = {cosine_similarity(a, c):.4f} (expected: 0 = cos(90°))")
print(f"cosine(a, d) = {cosine_similarity(a, d):.4f} (expected: -1 = cos(180°))")

# 驗證
print("\n✓ 驗證：")
assert np.isclose(dot_product(a, b), np.dot(a, b))
assert np.isclose(cosine_similarity(a, b), np.cos(np.pi/4))
print("所有測試通過！")

---
## Part 4: 矩陣乘法

矩陣乘法 `C = A @ B` 的規則：

- A 的 shape 是 (m, n)
- B 的 shape 是 (n, p)
- C 的 shape 是 (m, p)

$$C_{ij} = \sum_{k=1}^n A_{ik} B_{kj}$$

**直覺理解**：`C[i,j]` 是 A 的第 i **行**和 B 的第 j **列**的內積。

### 矩陣乘法的意義：線性變換

把 `y = Ax` 想成：矩陣 A 把向量 x「變換」成向量 y。

例如：
- **旋轉矩陣**可以旋轉向量
- **縮放矩陣**可以拉伸/壓縮向量
- **神經網路的全連接層**就是矩陣乘法 + 偏置

In [ ]:
# 示範：各種線性變換

def rotation_matrix(theta):
    """旋轉 theta 弧度"""
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s],
                     [s,  c]])

def scaling_matrix(sx, sy):
    """x 方向縮放 sx 倍，y 方向縮放 sy 倍"""
    return np.array([[sx, 0],
                     [0, sy]])

def shear_matrix(k):
    """剪切變換"""
    return np.array([[1, k],
                     [0, 1]])

# 原始的正方形頂點
square = np.array([[0, 0], [1, 0], [1, 1], [0, 1], [0, 0]]).T  # shape: (2, 5)

# 各種變換
transforms = [
    ('原始', np.eye(2)),
    ('旋轉 45°', rotation_matrix(np.pi/4)),
    ('縮放 (2, 0.5)', scaling_matrix(2, 0.5)),
    ('剪切', shear_matrix(0.5)),
]

plt.figure(figsize=(12, 3))
for i, (name, M) in enumerate(transforms):
    plt.subplot(1, 4, i+1)
    transformed = M @ square
    plt.plot(square[0], square[1], 'b--', alpha=0.5, label='原始')
    plt.fill(transformed[0], transformed[1], alpha=0.3)
    plt.plot(transformed[0], transformed[1], 'r-', linewidth=2)
    plt.xlim(-1, 2.5)
    plt.ylim(-0.5, 1.5)
    plt.grid(True, alpha=0.3)
    plt.gca().set_aspect('equal')
    plt.title(name)

plt.tight_layout()
plt.show()

### 練習 3: 實作矩陣乘法

**提示**：
- 用三層迴圈：外兩層遍歷 C 的每個元素 (i, j)，內層計算內積
- 或者用兩層迴圈 + numpy 的向量運算

In [ ]:
def matmul(A, B):
    """
    矩陣乘法 C = A @ B
    
    Parameters
    ----------
    A : np.ndarray, shape (m, n)
    B : np.ndarray, shape (n, p)
    
    Returns
    -------
    C : np.ndarray, shape (m, p)
    """
    m, n = A.shape
    n2, p = B.shape
    assert n == n2, f"Matrix dimensions don't match: {A.shape} and {B.shape}"
    
    # 解答（方法 1：三層迴圈）
    C = np.zeros((m, p))
    for i in range(m):
        for j in range(p):
            for k in range(n):
                C[i, j] += A[i, k] * B[k, j]
    return C


def matmul_v2(A, B):
    """
    矩陣乘法（更簡潔的寫法）
    
    使用 numpy 的向量運算，只需要兩層迴圈
    """
    m, n = A.shape
    n2, p = B.shape
    assert n == n2
    
    # 解答（方法 2：兩層迴圈 + 內積）
    C = np.zeros((m, p))
    for i in range(m):
        for j in range(p):
            # C[i,j] 是 A 的第 i 行和 B 的第 j 列的內積
            C[i, j] = np.sum(A[i, :] * B[:, j])
    return C

In [ ]:
# 測試
A = np.array([[1, 2],
              [3, 4]])
B = np.array([[5, 6],
              [7, 8]])

C_v1 = matmul(A, B)
C_v2 = matmul_v2(A, B)
C_numpy = A @ B

print("A:")
print(A)
print("\nB:")
print(B)
print("\nA @ B (你的實作):")
print(C_v1)
print("\nA @ B (numpy):")
print(C_numpy)

# 驗證
print("\n✓ 驗證：")
assert np.allclose(C_v1, C_numpy)
assert np.allclose(C_v2, C_numpy)
print("所有測試通過！")

---
## Part 5: 影像作為向量

在機器學習中，我們常常把影像「攤平」成向量：

- 一張 28×28 的灰階圖 → 784 維向量
- 一張 32×32×3 的彩色圖 → 3072 維向量

這樣就可以對影像做向量運算了！

### 為什麼這很重要？

1. **神經網路的輸入**：全連接層需要向量輸入
2. **計算相似度**：可以用 cosine similarity 比較兩張圖
3. **PCA 降維**：需要把影像當作高維向量

In [ ]:
# 創建一些簡單的「影像」
img1 = np.array([[0, 1, 2],
                 [3, 4, 5],
                 [6, 7, 8]])

print("Original image (3x3):")
print(img1)
print(f"Shape: {img1.shape}")

# 攤平成向量
vec = img1.flatten()  # 或 img1.reshape(-1) 或 img1.ravel()
print(f"\nFlattened vector: {vec}")
print(f"Shape: {vec.shape}")

# 可以再 reshape 回來
img_back = vec.reshape(3, 3)
print(f"\nReshaped back to image:\n{img_back}")

### 練習 4: 計算兩張影像的相似度

**提示**：
1. 把影像攤平成向量
2. 用你前面寫的 `cosine_similarity` 計算

In [ ]:
def image_similarity(img1, img2):
    """
    計算兩張影像的 cosine similarity
    
    1. 把影像攤平成向量
    2. 計算 cosine similarity
    """
    # 解答：
    vec1 = img1.flatten().astype(float)
    vec2 = img2.flatten().astype(float)
    return cosine_similarity(vec1, vec2)


def euclidean_distance(img1, img2):
    """
    計算兩張影像的歐幾里得距離
    """
    # 解答：
    vec1 = img1.flatten().astype(float)
    vec2 = img2.flatten().astype(float)
    return l2_norm(vec1 - vec2)

In [ ]:
# 測試
img_a = np.array([[1, 2], [3, 4]])
img_b = np.array([[1, 2], [3, 4]])  # 完全相同
img_c = np.array([[2, 4], [6, 8]])  # 比例相同（方向相同）
img_d = np.array([[4, 3], [2, 1]])  # 不同

print("Cosine Similarity:")
print(f"sim(a, b) = {image_similarity(img_a, img_b):.4f} (expected: 1.0，完全相同)")
print(f"sim(a, c) = {image_similarity(img_a, img_c):.4f} (expected: 1.0，方向相同)")
print(f"sim(a, d) = {image_similarity(img_a, img_d):.4f} (不同圖片)")

print("\nEuclidean Distance:")
print(f"dist(a, b) = {euclidean_distance(img_a, img_b):.4f} (expected: 0.0)")
print(f"dist(a, c) = {euclidean_distance(img_a, img_c):.4f}")
print(f"dist(a, d) = {euclidean_distance(img_a, img_d):.4f}")

---
## Summary

這個 notebook 你學到了：

1. **向量**可以表示點或方向，在 ML 中用來表示資料
2. **Norm** 是向量的「長度」，有 L1, L2, L∞ 等不同定義
3. **內積**可以測量兩向量的相似度（方向）
4. **Cosine Similarity** = 內積 / (兩個長度的乘積)，只看方向不看長度
5. **矩陣乘法**可以視為線性變換
6. **影像**可以攤平成向量來做運算

### 這些概念在 ML 中的應用：

| 概念 | 應用 |
|------|------|
| L2 Norm | 計算距離、L2 正則化 |
| L1 Norm | L1 正則化（產生稀疏解） |
| 內積 | 全連接層 y = Wx + b |
| Cosine Similarity | 文字/影像相似度、推薦系統 |
| 矩陣乘法 | 神經網路的核心運算 |

---

**下一個 notebook**: `02_gradients.ipynb` - 梯度與微分